In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

In [2]:
include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

load_ta_network (generic function with 2 methods)

In [3]:
using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

create_graph (generic function with 1 method)

In [4]:
function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

get_vector (generic function with 1 method)

In [5]:
ta_data = load_ta_network("East Massachusetts")

TA_Data("East Massachusetts",8,0,1,24,[1,2,1,3,2,3,2,4,3,5  …  5,6,4,8,5,7,6,7,7,8],[2,1,3,1,3,2,4,2,5,3  …  6,5,8,4,7,5,7,6,8,7],[1470.14,1431.18,1814.8,1838.48,1813.57,1842.59,1538.78,1536.4,1994.12,1958.68  …  1481.65,1193.67,1500.0,1500.0,1810.8,1822.92,1731.39,1672.85,1500.0,1500.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.223769,0.239099,0.335607,0.342556,0.1041,0.123493,0.393796,0.405647,0.176085,0.182724  …  0.126612,0.125992,0.251184,0.250322,0.136876,0.136113,0.186022,0.18427,0.143381,0.147352],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0],0.0,8x8 Array{Float64,2}:
   0.0          0.0       

In [6]:
# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

0.0

In [7]:
# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links)

8x8 sparse matrix with 24 Int64 entries:
	[2, 1]  =  2
	[3, 1]  =  4
	[1, 2]  =  1
	[3, 2]  =  6
	[4, 2]  =  8
	[1, 3]  =  3
	[2, 3]  =  5
	[5, 3]  =  10
	[6, 3]  =  12
	[2, 4]  =  7
	⋮
	[4, 5]  =  13
	[6, 5]  =  16
	[7, 5]  =  20
	[3, 6]  =  11
	[5, 6]  =  15
	[7, 6]  =  22
	[5, 7]  =  19
	[6, 7]  =  21
	[8, 7]  =  24
	[4, 8]  =  17
	[7, 8]  =  23

In [8]:
graph

Directed Graph (8 vertices, 24 edges)

In [9]:
coeffs_dict_Apr_AM = readall("../temp_files/coeffs_dict_Apr_AM.json")
coeffs_dict_Apr_AM = JSON.parse(coeffs_dict_Apr_AM)

Dict{String,Any} with 225 entries:
  "(5,1.0,0.1,1)"    => {0.09607510556659266,0.3450021316819924,0.5443004480578…
  "(6,1.0,1.0,1)"    => {0.049544514762758195,0.2334131544105789,0.483328618610…
  "(3,3.0,0.1,1)"    => {0.6492182976903935,0.4228723877486018,0.11501078808698…
  "(3,1.5,0.0001,1)" => {1.0691359364905215,-0.16586846430857405,0.373587976956…
  "(6,1.0,0.001,1)"  => {0.4369146360317604,0.24501115984458022,0.7727002425530…
  "(6,1.0,0.0001,1)" => {1.0246032346003422,-1.3479785115006955,0.9900956162562…
  "(6,2.0,10.0,1)"   => {0.19637967962593153,0.4577782314914942,0.4721142821578…
  "(6,2.0,0.01,1)"   => {1.06436506607719,-0.3064529411674317,0.438035415637049…
  "(5,1.5,1.0,1)"    => {0.1776967981628016,0.4557494512217777,0.49878922088773…
  "(6,1.0,1000.0,1)" => {0.04776909921954599,0.22973952133610526,0.481128541491…
  "(5,1.0,0.01,1)"   => {0.21812206497190748,0.5004123698549194,0.5370073503259…
  "(6,2.5,0.1,1)"    => {0.816363105080458,0.2850473926048943,0.1791488208

In [10]:
coeffs = coeffs_dict_Apr_AM["(7,1.5,0.1,1)"]

8-element Array{Any,1}:
  0.283888  
  0.54045   
  0.545002  
  0.167487  
 -0.108338  
 -0.104839  
 -0.0364901 
 -0.00346061

In [12]:
polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

polyEval (generic function with 1 method)

In [13]:
function BPR(x)
    # travel_time = free_flow_time .* ( 1.0 + B .* (x./capacity).^power )
    # generalized_cost = travel_time + toll_factor *toll + distance_factor * link_length
    # return generalized_cost

    bpr = similar(x)
    for i=1:length(bpr)
        bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
    end
    return bpr
end

BPR (generic function with 1 method)

In [14]:
function all_or_nothing(travel_time)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time, r)

        for s=1:size(travel_demand)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing (generic function with 1 method)

In [15]:
# Finding a starting feasible solution
travel_time = BPR(zeros(number_of_links))
xl = all_or_nothing(travel_time)

24-element Array{Float64,1}:
 2705.4  
  912.103
    0.0  
 1008.79 
 2705.4  
 1043.2  
 1378.19 
 1300.92 
 1564.8  
 1555.32 
 1071.23 
  219.497
  742.897
 1186.48 
 1043.35 
  933.357
 1102.06 
 1170.95 
 1253.92 
 2227.5  
  831.564
  611.977
 1253.44 
 1182.45 

In [23]:
max_iter_no = 1e12
l = 1
#average_excess_cost = 1
tol = 1e-5

while l < max_iter_no
    l += 1
    
    xl_old = xl
    
    # Finding yl
    travel_time = BPR(xl)
    
    yl = all_or_nothing(travel_time)
    
    xl = xl + (yl - xl)/l
    
    xl_new = xl
    
    relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

    if l % 10000 == 0
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
    end

    if relative_gap < tol 
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
        break
    end
    
end

l = 10000------------------------------------------------
relative_gap is 1.6553333164100392e-5
l = 10450------------------------------------------------
relative_gap is 9.994148466453076e-6


In [20]:
outfile = open("../temp_files/flows_converge_uni-class_case3.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], xl[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)

In [22]:
capacity

24-element Array{Float64,1}:
 1470.14
 1431.18
 1814.8 
 1838.48
 1813.57
 1842.59
 1538.78
 1536.4 
 1994.12
 1958.68
 1881.41
 1626.28
 1438.13
 1597.64
 1481.65
 1193.67
 1500.0 
 1500.0 
 1810.8 
 1822.92
 1731.39
 1672.85
 1500.0 
 1500.0 

In [21]:
xl

24-element Array{Float64,1}:
 2705.4  
  912.103
    0.0  
 1008.79 
 2705.42 
 1201.82 
 1378.17 
 1142.29 
 1441.14 
 1120.55 
 1194.91 
  812.894
  901.529
 1186.5  
 1043.36 
  933.357
 1102.06 
 1170.96 
 1130.25 
 1634.09 
  955.241
 1205.37 
 1253.44 
 1182.44 

In [41]:
c_grid = -5.:.5:5

-5.0:0.5:5.0

In [42]:
deg_grid = [2:6]

5-element Array{Int64,1}:
 2
 3
 4
 5
 6